In [ ]:
using Catalyst, DifferentialEquations
using BlackBoxOptim
using CSV
using DataFrames
using DataStructures
using Statistics
using NaNStatistics
using Plots
using Latexify
include("../src/IRFConvolution.jl") 
include("../src/DataImport.jl")
include("../src/TypeDefinitions.jl")
include("../src/KineticModel.jl") 


## Define your kinetic model and parameter bounds
*Only edit this cell*

In [ ]:
# import data
RealData = importData("/Users/jessicaflowers/Desktop/JULIA/data/Fe2O3/"; miss="NaN")
# RealData = RealData[1]

# define kinetic model
rs = @reaction_network begin
    k1, A --> 0
    k2, 2B --> 0
    k3, C --> 0
end

# odesys = convert(ODESystem, rs) # print the differential equations for this reaction system
# latexify(rs) # print the reaction system

# define bounds for the parameter optimization 
state_lower = [1.0e6, 8e4, 1.0e6] # A, B, C
state_upper = [1.0e6, 8e6, 1.0e6] 

IRF_lower = [0.1, 0.01] # μ, σ
IRF_upper = [0.2, 0.08]

rate_const_lower = [0.001, 1e-7, 1e-8] # k1, k2, k3, ...
rate_const_upper = [0.01, 1e-5, 1e-6]

### Pre-processing of the input csv file

In [ ]:
# Extract wavelength range and time vector, wavelength range is the first column and time vector is the first row
wavelength = RealData.x
wavelength = vcat(wavelength...)
time = RealData.y
time = vcat(time...)
Data = RealData.z
Data = (vcat(Data...))
Data = coalesce.(Data, NaN)


# create bounds array
lower = vcat(state_lower, IRF_lower, rate_const_lower)
upper = vcat(state_upper, IRF_upper, rate_const_upper)

bounds = Array{Tuple{Float64, Float64}}(undef,length(lower))
for k in 1:length(lower)
    bounds[k] = (lower[k], upper[k])
end

### Run the optimization

In [ ]:
OP = run_optim(Objective, bounds)

## Data processing and plotting

In [ ]:
RecoveredParam = best_candidate(OP) # recover optimized parameter vector 
RecoveredData = GetData(RecoveredParam) # plug optimized parameters into GetData
DataMatrix = RecoveredData[1] # data matrix
RecoveredSpec = RecoveredData[2] # spectral signatures
RecoveredKinetics = RecoveredData[3] # kinetic

In [ ]:
# kinetics
tpos = time.>0
Plots.plot(time[tpos], RecoveredKinetics[1,:][tpos], xscale=:log10,
     title="Recovered Kinetics", xlabel="log(Time)", legend=:topleft, ylabel="Population",
     linewidth = 2, xguidefontsize=10, yguidefontsize=10, label="A(t)")
Plots.plot!(time[tpos], RecoveredKinetics[2,:][tpos], xscale=:log10,
     linewidth = 2, xguidefontsize=10, yguidefontsize=10, label="B(t)")
Plots.plot!(time[tpos], RecoveredKinetics[3,:][tpos], xscale=:log10,
linewidth = 2, xguidefontsize=10, yguidefontsize=10, label="C(t)")


In [ ]:
# Spectral Signatures

Plots.plot(wavelength, RecoveredSpec[:,1], title="Recovered Spectral Signatures", xlabel="Wavelength",
linewidth = 2, xguidefontsize=10, yguidefontsize=10, label="A(t)", ylabel="Δ Absorbance" ) 
Plots.plot!(wavelength, RecoveredSpec[:,2], title="Recovered Spectral Signatures", xlabel="Wavelength",
linewidth = 2, xguidefontsize=10, yguidefontsize=10, label="B(t)" ) 
Plots.plot!(wavelength, RecoveredSpec[:,3], title="Recovered Spectral Signatures", xlabel="Wavelength",
linewidth = 2, xguidefontsize=10, yguidefontsize=10, label="C(t)" ) 

In [ ]:
# residual maps 
# Map = Objective(RecoveredParam; output="map") # why does this not work anymore?
# resMap(RecoveredParam)
Map = heatmap((DataMatrix .- Data))
residual_value = Objective(RecoveredParam)


In [ ]:
# heatmap of real data
real_heatmap = heatmap(time, wavelength, Data, xlabel="Time (ps)", ylabel="Wavelength (nm)", title="Experimental Data", 
    colorbar_title="\n\nΔ Absorbance", right_margin=15Plots.mm, left_margin=10Plots.mm,
    xguidefontsize=10, yguidefontsize=10, c=:thermal)


In [ ]:
# heatmap of recovered data
optim_heatmap = heatmap(time, wavelength, DataMatrix, xlabel="Time", ylabel="Wavelength", 
    title="Recovered Data" ,colorbar_title="\n\nΔ Absorbance", right_margin=15Plots.mm, 
    left_margin=10Plots.mm, xguidefontsize=10, yguidefontsize=10, c=:thermal)
